This is a small and quick project for understanding the basics of RAGing using local running LLMs(more specifically Llama in this case) with Ollama and langchain.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPEN_API_KEY")

#MODEL = "gpt-3.5-turbo"
# MODEL = "mixtral:8x7b"
MODEL = "llama3.2:1b"

In [2]:
#from langchain_openai.chat_models import ChatOpenAI
#from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings


#model = ChatOpenAI(api_key=OPENAI_API_KEY, model=MODEL)
#embeddings = OpenAIEmbeddings()
model = OllamaLLM(model=MODEL)
embeddings = OllamaEmbeddings(model = MODEL)

model.invoke("tell me a joke")

'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."'

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" The librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."'

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'mlschool.pdf', 'page': 0}, page_content='Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine\nlearning systems from the ground up.\nNext cohort:\xa0November4 - 21, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and finding\nhelp to solve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models\nfor some of the largest companies in the world. I want to show you how to do the\nsame.\nThis is the class I wish I had taken when I started."This is the best machine learning course I\'ve done.\nWorth every cent."\n

In [17]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [18]:
chain = prompt | model | parser


In [15]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [21]:
chain.invoke({
    "context": "My name is Santiago.",
    "question": "What is my name?"
})

"Since this conversation has just started and I don't have any prior information about you or your identity, I will reply accordingly.\n\nYour name is Santiago."

In [22]:
chain.invoke({
    "context": "The name I was given was Santiago",
    "question": "What is my mom's name?"
})

"Since I don't have any information about a person named Santiago other than their given name, I wouldn't be able to determine who their mom might be."

In [23]:
from langchain_community.vectorstores import DocArrayInMemorySearch # For local testing it is enough, for a real app you would want a persistent vector storage

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding= embeddings
)

In [25]:
retriever = vectorstore.as_retriever() # A retriever is a component, from LangChain, that allows you to retrieve information from basically anywhere. but in this case, from a vector store. 

print(retriever.invoke("Machine learning")) # This should retrieve the most significant vectors containing information about a giver search term

[Document(metadata={'source': 'mlschool.pdf', 'page': 6}, page_content="Program Syllabus\nThis program will teach you the practical skills and insights that will\nhelp you build machine learning systems.\nHere are the contents of the six live sessions of the program:\nSession 1 - How To Start (Almost) Any Project\nWhat makes production machine learning different from what you've learned.\nThe strategy to solve the right problem using the right solution.\nCritical questions to ask before starting any project.\nProblem framing, inversion, and the haystack principle for building successful\napplications.\nThe first rule of machine learning engineering and how to start building.\nData collection strategies. A technique to determine how much data you\nneed.\nThe problem of selection bias and how to deal with it.\nLabeling data. Human annotations, natural labels and weak supervision.\nActive learning using the uncertainty and diversity sampling strategies.\nSession 2 - How to Build a Model\n

In [31]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "What is machine learning?"})

"I don't know. The document doesn't explicitly define what machine learning is, but it does provide some context about its relevance to real-world problems and the field of software engineering."

In [33]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the course?
Answer: Based on the context, it appears that the purpose of the course is to provide technical professionals with a practical and hands-on training in machine learning systems. The document outlines various aspects of the course, such as:

* Live sessions for coding instructions
* Step-by-step coding examples
* A final project where students will build an end-to-end system from scratch
* Lifetime access to past and future cohorts, allowing students to continue learning without interruption

The document also mentions that the program is designed to help technical professionals "unlearn what you think machine learning is" by providing a hands-on approach to learning.

Question: How many hours of live sessions?
Answer: According to the document, you have lifetime access to every past and future cohort. You can join any iteration of the program at any time without waiting for a specific cohort to join.

However, the document does mention that 

In [34]:
for s in chain.stream({"question": "What is the purpose of the course?"}): # streams character by character the model generates
    print(s, end="", flush=True)

Based on the context, it appears that the purpose of the course is to teach machine learning concepts and provide hands-on training for technical professionals who want to build production systems using Python. The document mentions "a practical, hands-on class where you'll learn from years of experience and real-world examples", suggesting that the course aims to help students gain practical knowledge in machine learning through interactive sessions, coding instructions, and projects.

In [35]:
chain.batch([{"question": q} for q in questions]) # performs all questions in parallel

['The purpose of the course is to help students unlearn what they think machine learning is and learn from years of experience and real-world examples.',
 'The document states that there will be 2 hours of live sessions per Monday, 1 hour of individual work on Tuesday, and 2 hours of live sessions per Thursday. There are also 1 hour of individual work on Friday. \n\nTo find the total number of hours of live sessions, we need to calculate the sum:\n\n2 hours (Monday) + 1 hour (Tuesday) + 2 hours (Thursday) = 5 hours\n+ 1 hour (Friday) = 6 hours\n\nSo, there will be a total of 6 hours of live sessions.',
 'There is no mention of coding assignments in the provided context. The text mentions live sessions, data parallelism, model parallelism, and code walkthroughs, but it does not specify any coding assignments as part of the program.',
 'I don\'t know. The document doesn\'t explicitly state that there will be a program certificate upon completion, but it does mention "lifetime access" whi